In [2]:
import csv
import os

def write_csv(data, file_name):

    with open(file_name, 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',')
        for row in data:
            spamwriter.writerow(row)

def read_csv(file_name):

    data = []
    with open(file_name) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            data.append(row)
    return data

# after = []
# for fold in os.listdir("./outputs/"):
#     for file in os.listdir(os.path.join("./outputs/", fold)):
#         if "afterPost" in file:
#             after.append(file)
#             file_path = os.path.join(os.path.join("./outputs/", fold, file))
#             os.system(f"cp -r {file_path} ./output_all")
#             print(file)
# print(len(after))

In [7]:
# get run time
import re
from dateutil.parser import parse

def extract_dates_times(string):
    pattern = r'\d{2}:\d{2}:\d{2}'
    matches = re.findall(pattern, string)
    dates_times = []

    for match in matches:
        dates_times.append(match)
    return dates_times

from datetime import datetime, timedelta

def time_to_seconds(time_string):
    dt = datetime.strptime(time_string, "%H:%M:%S")
    td = timedelta(hours=dt.hour, minutes=dt.minute, seconds=dt.second)
    seconds = td.total_seconds()
    return seconds


after = list(os.listdir("./output_all/"))
# after = [".".join(name.split(".")[3:-1]) for name in after]
# print(len(after))
run_time = dict()
for file in after:
    pcb_name = ".".join(file.split(".")[3:-1])

    log_file = f"./all_logs/{pcb_name}.kicad_pcb.log"
    with open(log_file, 'r') as file:
        log = file.read().split('\n')[-2]
    time = extract_dates_times(log)[0]
    time = time_to_seconds(time)
    run_time[pcb_name] = time
print(run_time)



{'TMC261-stepstick_TMC261-stepstick-v1.1': 198.0, 'Inkjet__autosave-InkjetDriver': 9.0, 'domotics_in-board': 8.0, 'espalarm_alarm': 43.0, 'kitspace_OutlineChaos': 23.0, 'Hardware_Playground_Touch_Switch_2ch_PCB': 101.0, 'hardware-designs_energy-harvester': 19.0, 'kitspace_piezo_amplifier': 92.0, 'imfr-schematics_Telescopio': 41.0, 'kitspace_USBI2C01': 26.0, 'Inkjet_InkjetBreakout': 2.0, 'deskbot_breakout': 10.0, 'kitspace_Potentiometer_mount_4LED': 0.0, 'xwhatits-capsense-controller_model-f-3178-adaptor': 0.0, 'ID-FIX_scanConnect': 0.0, 'z2amiller_sensorboard_programmer': 0.0, 'PiZeroHub_PiZeroHub': 52.0, 'bmw-ibus-bluetooth_bmw_bt_cdcemu_analog': 2.0, 'Shift-out-32-HC595_Shift-out': 22.0, 'opentilt_opentilt2': 3.0, 'OpenHardwareExG_Shield_OpenHardwareExG_Shield_Test_Board_all_panelled': 504.0, 'nanoTracer_nanoTracer': 81.0, 'gepetto_circuito': 12.0, 'bldc-gimbal-1d_gimbal-board': 7.0, 'KiwiSDR_PCB_ant': 227.0, 'LoLin_Designs_LoLin': 2.0, 'MySRaspiGW_MySRaspiGW_PA_LNA_Pimoroni': 1.0, '

In [11]:
print(len(after))

57


In [10]:
import os
from scipy.spatial.distance import euclidean

from Scripts.Data_extraction.thirdparty.kicad_parser.kicad_pcb import *
from Scripts.Data_extraction.thirdparty.kicad_parser.sexp_parser import *


after = list(os.listdir("./output_all/"))
wirelength = dict()
via = dict()
for pname in after:
    # print(pname)
    pcb = KicadPCB.load('./output_all/' + pname)
    wl = 0
    for seg in pcb.segment:
        wl += euclidean(seg.start, seg.end)
    # if len(pname.split(".")[-2]) != 1:
    #     pcb_name = pname.split(".")[-2]
    # else:
    #     pcb_name = ".".join(pname.split(".")[-3:-1])
    pcb_name = ".".join(pname.split(".")[3:-1])
    wirelength[pcb_name] = wl
    via[pcb_name] = len(pcb.via)

print(via)
print(wirelength)


{'TMC261-stepstick_TMC261-stepstick-v1.1': 209, 'Inkjet__autosave-InkjetDriver': 39, 'domotics_in-board': 2, 'espalarm_alarm': 55, 'kitspace_OutlineChaos': 24, 'Hardware_Playground_Touch_Switch_2ch_PCB': 97, 'hardware-designs_energy-harvester': 14, 'kitspace_piezo_amplifier': 75, 'imfr-schematics_Telescopio': 7, 'kitspace_USBI2C01': 24, 'Inkjet_InkjetBreakout': 6, 'deskbot_breakout': 17, 'kitspace_Potentiometer_mount_4LED': 0, 'xwhatits-capsense-controller_model-f-3178-adaptor': 0, 'ID-FIX_scanConnect': 0, 'z2amiller_sensorboard_programmer': 2, 'PiZeroHub_PiZeroHub': 41, 'bmw-ibus-bluetooth_bmw_bt_cdcemu_analog': 9, 'Shift-out-32-HC595_Shift-out': 47, 'opentilt_opentilt2': 7, 'OpenHardwareExG_Shield_OpenHardwareExG_Shield_Test_Board_all_panelled': 139, 'nanoTracer_nanoTracer': 25, 'gepetto_circuito': 14, 'bldc-gimbal-1d_gimbal-board': 43, 'KiwiSDR_PCB_ant': 41, 'LoLin_Designs_LoLin': 1, 'MySRaspiGW_MySRaspiGW_PA_LNA_Pimoroni': 5, 'audprog_audprog_v2': 40, 'xwhatits-capsense-controller_

In [11]:
data = read_csv("./Results_FR.csv")
for i, row in enumerate(data):
    if row[0] in run_time and row[5] == "":
        row[5] = "Yes"
        row[6] = wirelength[row[0]]
        row[7] = via[row[0]]
        row[8] = run_time[row[0]] if float(run_time[row[0]]) > 0 else 0.1

        data[i] = row
    elif row[0] not in run_time and row[5] == "":
        row[5] = "No"

write_csv(data, "Results_FR_PcbRouter.csv")